In [1]:
# Import required modules
from copy import copy

import numpy as np
import time

from lcode2dPy.plasma3d.initialization import init_plasma
from lcode2dPy.plasma3d.solver import Plane2d3vPlasmaSolver 

from lcode2dPy.config.default_config import default_config

In [2]:
# Create a config
config = copy(default_config)

config.set('geometry', '3d')

config.set('window-width-step-size', 0.02)
config.set('window-width', 15.38)

config.set('window-length', 3000)
config.set('xi-step', 0.02)

config.set('plasma-particles-per-cell', 4)

# Make ready a plasma solver
solver = Plane2d3vPlasmaSolver(config)

In [3]:
# Create a transversal grid and a beam charge distribution
grid_steps     = config.getint('window-width-steps')
grid_step_size = config.getfloat('window-width-step-size')

grid = ((np.arange(grid_steps) - grid_steps // 2)
        * grid_step_size)
xs, ys = grid[:, None], grid[None, :]

def rho_b(xi):
    COMPRESS, BOOST, SIGMA, SHIFT = 1, 1, 1, 0
    if xi < -2 * np.sqrt(2 * np.pi) / COMPRESS or xi > 0:
        return 0.
        # return np.zeros_like(xs)
    r = np.sqrt(xs**2 + (ys - SHIFT)**2)
    return (.05 * BOOST * np.exp(-.5 * (r / SIGMA)**2) *
            (1 - np.cos(xi * COMPRESS * np.sqrt(np.pi / 2))))

In [ ]:
# Create the first plasma slice
fields, particles, currents, const_arrays = init_plasma(config)

# Simulation loop:
start_time = time.time()
xi_step_size = config.getfloat('xi-step')
xi_steps = int(config.getfloat('window-length') / xi_step_size)
ez = []

for xi_i in range(xi_steps + 1):
    xi =  - xi_i * xi_step_size
    
    ro_beam = rho_b(xi)
    ro_beam_prev = rho_b(xi + xi_step_size)

    particles, fields, currents = solver.step_dxi(
        particles, fields, currents, const_arrays, ro_beam, ro_beam_prev
    )

    ez.append(fields.Ez[grid_steps // 2, grid_steps // 2])
    print(f'xi={xi:+.4f} {ez[-1]:+.4e}')
    
print("--- %s seconds ---" % (time.time() - start_time))

In [11]:
# Should work correctly. Compare results with:
# xi         Ez_00
# xi=+0.0000 +0.0000e+00
# xi=+0.0200 -1.4490e-07
# xi=+0.0400 -7.2443e-07
# xi=+0.0600 -2.0278e-06
# xi=+0.0800 -4.3436e-06
# xi=+0.1000 -7.9592e-06
# xi=+0.1200 -1.3161e-05
# xi=+0.1400 -2.0233e-05
# xi=+0.1600 -2.9458e-05
# xi=+0.1800 -4.1116e-05
# xi=+0.2000 -5.5486e-05
# xi=+0.2200 -7.2842e-05
# xi=+0.2400 -9.3456e-05
# xi=+0.2600 -1.1760e-04
# xi=+0.2800 -1.4553e-04
# xi=+0.3000 -1.7751e-04
# xi=+0.3200 -2.1381e-04
# xi=+0.3400 -2.5467e-04
# xi=+0.3600 -3.0033e-04
# xi=+0.3800 -3.5105e-04
# xi=+0.4000 -4.0706e-04
# xi=+0.4200 -4.6859e-04
# xi=+0.4400 -5.3587e-04
# xi=+0.4600 -6.0912e-04
# xi=+0.4800 -6.8855e-04
# xi=+0.5000 -7.7437e-04